In [2]:
pip install gensim

     |████████████████████████████████| 24.0 MB 7.0 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 13.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import gensim
from gensim.models import Word2Vec, KeyedVectors
import logging
import time
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import requests
import matplotlib.pyplot as plt
import gensim.downloader as api # Downloading the Google pretrained Word2Vec Model

In [3]:
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

/home/vkanoria/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [4]:
start_time = time.time() #Start the timer
w2v_model = KeyedVectors.load_word2vec_format(path, binary=True) #load the model
print("%0.2f seconds taken to load"%float(time.time() - start_time)) #Calculate the total time elapsed since starting the timer
print('-'*10)

print('Finished loading Word2Vec')
print('-'*10)

98.55 seconds taken to load
----------
Finished loading Word2Vec
----------


In [5]:
#check to see if the word2vec model works
#w2v_model.most_similar('beautiful')

In [6]:
df = pd.read_csv('unique_books.csv')

In [7]:
df.head()

,Unnamed: 0,user_id,book_id,rating,SortIndex1,SortIndex2,goodreads_book_id,isbn,isbn13,original_title,average_rating,ratings_count,book_authors,book_desc,book_isbn,book_title,genres,book_desc_tok
0,0,2,2318,3,41929,398,998.0,671015206,9.780671e+12,The Millionaire Next Door: The Surprising Secr...,4.00,43937.0,"['Thomas J. Stanley', 'William D. Danko']",The incredible national bestseller that is cha...,9.780670e+12,The Millionaire Next Door: The Surprising Secr...,"['Economics', 'Finance', 'Nonfiction', 'Busine...",the incredible national bestseller ...
1,389,2,26,4,41929,227,968.0,307277674,9.780307e+12,The Da Vinci Code,3.79,1447148.0,['Dan Brown'],An ingenious code hidden in the works of Leona...,9.780310e+12,The Da Vinci Code,"['Fiction', 'Mystery', 'Thriller']",an ingenious code hidden in t...
2,13461,2,315,3,41929,193,4894.0,91883768,9.780092e+12,Who Moved My Cheese?,3.74,237272.0,"['Spencer Johnson', 'Kenneth H. Blanchard']","Written by Spencer Johnson, coauthor of The On...",9.780090e+12,Who Moved My Cheese?,"['Business', 'Self Help', 'Nonfiction', 'Psych...",written by spencer johnson ...
3,15287,2,33,4,41929,98,930.0,739326228,9.780739e+12,Memoirs of a Geisha,4.08,1300209.0,['Arthur Golden'],"A literary sensation and runaway bestseller, t...",9.780740e+12,Memoirs of a Geisha,"['Fiction', 'Historical', 'Historical Fiction'...",a literary sensation and runaway...
4,49199,2,301,5,41929,194,4900.0,1892295490,9.781892e+12,Heart of Darkness,3.42,255576.0,['Joseph Conrad'],Dark allegory describes the narrator's journey...,9.781890e+12,Heart of Darkness,"['Classics', 'Fiction', 'Literature', 'Cultura...",dark allegory describes the narr...


In [8]:
corpus = []
for desc in df['book_desc_tok']:
    if type(desc)==str:
        corpus.append(desc.split())

In [9]:
corpus

[['the',
  'incredible',
  'national',
  'bestseller',
  'that',
  'is',
  'changing',
  'people',
  's',
  'lives',
  'and',
  'increasing',
  'their',
  'net',
  'worth',
  'can',
  'you',
  'spot',
  'the',
  'millionaire',
  'next',
  'door',
  'who',
  'are',
  'the',
  'rich',
  'in',
  'this',
  'country',
  'what',
  'do',
  'they',
  'do',
  'where',
  'do',
  'they',
  'shop',
  'what',
  'do',
  'they',
  'drive',
  'how',
  'do',
  'they',
  'invest',
  'where',
  'did',
  'their',
  'ancestors',
  'come',
  'from',
  'how',
  'did',
  'they',
  'get',
  'rich',
  'can',
  'i',
  'ever',
  'become',
  'one',
  'of',
  'them',
  'get',
  'the',
  'answers',
  'in',
  'the',
  'millionaire',
  'next',
  'door',
  'the',
  'never',
  'before',
  'told',
  'story',
  'about',
  'wealth',
  'in',
  'america',
  'you',
  'll',
  'be',
  'surprised',
  'at',
  'what',
  'you',
  'find',
  'out'],
 ['an',
  'ingenious',
  'code',
  'hidden',
  'in',
  'the',
  'works',
  'of',
  'l

In [10]:
len(corpus)

4327

In [11]:
google_model = Word2Vec(vector_size = 768, window=5, min_count = 2, workers = -1, epochs=40)
google_model.build_vocab(corpus)

In [12]:
google_model.train(corpus, total_examples=google_model.corpus_count, epochs = 10)

(0, 0)

## Average Word2Vec calculation for each book:

In [13]:
def generate_avg_word_vectors(x):
    
    # Creating a list for storing the vectors (description into vectors)
    global word_embeddings
    word_embeddings = []

    # Reading the each book description 
    for line in df['book_desc_tok']:
        avgword2vec = None
        count = 0
        for word in line.split():
            if word in google_model.wv:
                count += 1
                if avgword2vec is None:
                    avgword2vec = google_model.wv[word]
                    #add_vector
                else:
                    avgword2vec = avgword2vec + google_model.wv[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
        
            word_embeddings.append(avgword2vec)
    
    return word_embeddings
   

In [14]:
avg_word_vectors =  generate_avg_word_vectors(df)

#pd.DataFrame.from_dict({'title':})

In [15]:
avg_word_vectors[0]

array([ 1.90243827e-05, -1.70359781e-05,  1.27843916e-04, -1.88918747e-07,
       -1.73503053e-04,  2.82845594e-05,  2.01359027e-04,  1.37891810e-04,
        4.79418086e-05,  1.13530550e-05,  2.76593986e-04, -1.82160496e-04,
       -6.62505045e-05, -1.69062150e-05,  1.08471613e-05,  5.37750702e-06,
       -1.44549515e-04,  4.33733403e-05, -1.63556891e-04, -1.95378361e-05,
        1.63819466e-04,  7.37708760e-05,  1.28147542e-04, -8.43755843e-05,
       -8.86000926e-05,  6.49543726e-05,  1.48426279e-05,  3.22033593e-05,
       -3.34939650e-05, -1.06222767e-04,  5.48196613e-06,  1.55179605e-05,
        3.11695476e-05,  2.11202787e-05, -1.66503669e-04, -2.27439546e-06,
        2.67076190e-04, -4.22552366e-05,  5.74316691e-05, -2.38168050e-05,
        7.10145177e-05, -1.68517465e-04, -2.91583874e-05, -8.76910126e-05,
        6.42595887e-06,  1.94473687e-05, -1.24284750e-04,  1.26968662e-04,
        1.26454615e-04, -1.91060153e-05, -9.22650315e-07, -3.33724420e-05,
       -2.10949904e-04, -

In [16]:
#save avg_word_vectors as csv
avg_word_vectors_df = pd.DataFrame(avg_word_vectors)

In [17]:
avg_word_vectors_df.shape

(4327, 768)

In [18]:
avg_word_vectors_df.to_csv('book_embeddings_from_avg_word_vectors.csv')

important source:
https://towardsdatascience.com/3-types-of-contextualized-word-embeddings-from-bert-using-transfer-learning-81fcefe3fe6d